This Notebook is to compute the mean image of meteorology data. 

Specifically, we:<br>
1) stack 3-years of meteorology data;<br>
2) create the mean image;<br>
3) export the mean image to Assest

In [1]:
import ee
import datetime
import os
import itertools
import sys
import re

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

##### Define basic parameters

In [6]:
# define the years to be classified
year_name  = [f'{i}_{i+2}' for i in range(1990,2020,3)]

# import the research area boundary
North_China_Plain = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain/0_1_North_China_Plain_Full")

##### Stack meterology img of each 3-year, compute the mean value and export the mean_img

In [18]:
# prepare the climate data
Climate_type = ['lrad','prec','pres','shum','srad','temp','wind']

for year in range(1990,2019,3):   
    
    # define the three years according to year-variable
    year_start = year
    year_mid   = year + 1
    year_end   = year + 2
    
    # Initiate an empty list to hold the climate-tifs-mean from year_start to year_end
    climate_tifs_mean = []
    
    # compute the mean-tif of each climate-type
    for i in Climate_type:

        # Because cliate_data doesn't include data of 2019, so 
        # use if-else judgement to exclede the data of year 2019
        if year < 2017:
            climate_tifs = [ee.Image(f"users/wangjinzhulala/North_China_Plain_Python/Climate_data/{i}/{i}_{t}")                        
                            for t in [year_start,year_mid,year_end]]
            tifs_mean    = ee.ImageCollection(climate_tifs).mean().toFloat().rename(i)

        else:
            climate_tifs = [ee.Image(f"users/wangjinzhulala/North_China_Plain_Python/Climate_data/{i}/{i}_{t}")                        
                            for t in [year_start,year_mid]]

            tifs_mean    = ee.ImageCollection(climate_tifs).mean().toFloat().rename(i)

        # add all climate_tifs_mean of year_start to year_end to list
        climate_tifs_mean.append(tifs_mean)
        climate_tifs_mean_img = ee.Image(climate_tifs_mean)
        

    # export the mean_img to GEE asset
    asset_path  = f'users/wangjinzhulala/North_China_Plain_Python/Climate_data'
    export_name = f'Mean_{year}_{year + 2}'
    
    # create the export task
    task = ee.batch.Export.image.toAsset(   image          = climate_tifs_mean_img,
                                            description    = export_name,
                                            assetId        = f'{asset_path}/{export_name}',
                                            region         = North_China_Plain.geometry().bounds(),
                                            scale          = 10000,
                                            maxPixels      = int(1e13)) 
    # start the task
    task.start()
    
    # print out the process
    print(f'Exporting {export_name} successful!')

Exporting Mean_1990_1992 successful!
Exporting Mean_1993_1995 successful!
Exporting Mean_1996_1998 successful!
Exporting Mean_1999_2001 successful!
Exporting Mean_2002_2004 successful!
Exporting Mean_2005_2007 successful!
Exporting Mean_2008_2010 successful!
Exporting Mean_2011_2013 successful!
Exporting Mean_2014_2016 successful!
Exporting Mean_2017_2019 successful!
